In [2]:
import os
import zipfile
import shutil

with zipfile.ZipFile('/content/drive/MyDrive/Datasets/SOURCE2/detection/xml.zip', 'r') as f:
  f.extractall('/content/')

In [6]:
test_ids = []
with open('/content/drive/MyDrive/Datasets/SOURCE2/splits/test.uttlist', 'r') as split_f:
  test_ids = [line.strip().replace('\n', '') for line in split_f.readlines()]

train_ids = []
with open('/content/drive/MyDrive/Datasets/SOURCE2/splits/train.uttlist', 'r') as split_f:
  train_ids = [line.strip().replace('\n', '') for line in split_f.readlines()]

validation_ids = []
with open('/content/drive/MyDrive/Datasets/SOURCE2/splits/validation.uttlist', 'r') as split_f:
  validation_ids = [line.strip().replace('\n', '') for line in split_f.readlines()]

print(test_ids)
print(train_ids)
print(validation_ids)

['c04-110', 'c04-116', 'c04-134', 'c04-139', 'c04-144', 'c04-150', 'c04-165', 'c04-170', 'c06-011', 'd01-016', 'd01-019', 'd01-049', 'd01-052', 'd01-080', 'd01-085', 'd01-098', 'd01-104', 'd01-118', 'd01-123', 'd03-117', 'd04-012', 'd04-016', 'd04-021', 'd04-028', 'd04-032', 'd04-037', 'd04-047', 'd04-050', 'd04-053', 'd04-058', 'd04-062', 'd04-066', 'd04-086', 'd04-089', 'd04-096', 'd04-101', 'd04-111', 'd04-125', 'd04-131', 'd05-008', 'd05-013', 'd05-021', 'd05-025', 'd05-030', 'd05-040', 'd06-003', 'd06-008', 'd06-011', 'd06-020', 'd06-025', 'd06-027', 'd06-037', 'd06-041', 'd06-046', 'd06-056', 'd06-060', 'd06-063', 'd06-067', 'd06-072', 'd06-076', 'd06-086', 'd06-096', 'd06-100', 'd06-104', 'd06-107', 'd06-111', 'd06-113', 'd07-082', 'd07-085', 'd07-089', 'd07-093', 'd07-096', 'd07-100', 'd07-102', 'e01-055', 'e06-000', 'e06-003', 'e06-010', 'e06-015', 'e06-021', 'e06-026', 'e06-030', 'e06-033', 'e06-046', 'e06-049', 'e06-053', 'e06-070', 'f04-032', 'f04-035', 'f04-039', 'f04-043'

In [15]:
xml_filepaths = [os.path.join('/content/xml', f) \
                 for f in os.listdir('/content/xml') \
                 if os.path.isfile(os.path.join('/content/xml', f))]

test_xml_files = [f for f in xml_filepaths \
                  if os.path.basename(f)[:-4] in test_ids]

train_xml_files = [f for f in xml_filepaths \
                   if os.path.basename(f)[:-4] not in test_ids \
                   and os.path.basename(f)[:-4] not in validation_ids]

validation_xml_files = [f for f in xml_filepaths \
                        if os.path.basename(f)[:-4] in validation_ids]

print(len(xml_filepaths),
      len(test_xml_files) + len(train_xml_files) + len(validation_xml_files),
      len(test_ids) + len(train_ids) + len(validation_ids))

1539 1539 1199


Counts seems to be fine, so we can proceed and extract final results.

In [18]:
out_dir = '/content/SOURCE2/'

os.makedirs(out_dir, exist_ok=True)

import xml.etree.ElementTree as ET

def parse_iam_annotation(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    word_info = []

    for handwritten_part in root.findall(".//handwritten-part"):
        for line in handwritten_part.findall("line"):
            for word in line.findall("word"):
                word_id = word.attrib['id']
                transcription = word.attrib['text']

                word_info.append({
                    "word_id": word_id,
                    "transcription": transcription,
                })

    result = {
        "words": word_info,
    }

    return result




In [23]:
import json

test_annotation_data = {}

for xml_p in test_xml_files:
  id = os.path.basename(xml_p)[:-4]
  parsed_data = parse_iam_annotation(xml_p)
  test_annotation_data[id] = {"words": parsed_data['words']}

with open('/content/SOURCE2/test_annotations.json', 'w') as f:
  json.dump(test_annotation_data, f)

In [24]:
train_annotation_data = {}

for xml_p in train_xml_files:
  id = os.path.basename(xml_p)[:-4]
  parsed_data = parse_iam_annotation(xml_p)
  train_annotation_data[id] = {"words": parsed_data['words']}

with open('/content/SOURCE2/train_annotations.json', 'w') as f:
  json.dump(train_annotation_data, f)

In [27]:
validation_annotation_data = {}

for xml_p in validation_xml_files:
  id = os.path.basename(xml_p)[:-4]
  parsed_data = parse_iam_annotation(xml_p)
  validation_annotation_data[id] = {"words": parsed_data['words']}

with open('/content/SOURCE2/validation_annotations.json', 'w') as f:
  json.dump(validation_annotation_data, f)